In [1]:
import pandas as pd
# Check for Missing values
capacity = pd.read_excel('NIU Data Visualization Competition.xlsx',sheet_name = 'capacity')
print(capacity.isna().sum())
print(capacity.shape)
# Check for Duplicate records
duplicate_count = capacity.duplicated(keep='first').sum()
print(f"\nDuplicate Records:{duplicate_count}")
capacity=capacity.drop_duplicates()

print(capacity.shape)

geonode_id                   0
sku_id                       0
start_date                   0
standard_capacity            0
max_capacity                 0
num_of_standard_prod_days    0
num_of_max_prod_days         0
dtype: int64
(22230, 7)

Duplicate Records:234
(21996, 7)


In [2]:
# Check for Missing values
demand_forecast = pd.read_excel('NIU Data Visualization Competition.xlsx',sheet_name = 'demand forecast')
print(demand_forecast.isna().sum())

# Check for Duplicate records
duplicate_count = demand_forecast.duplicated(keep='first').sum()
print(f"\nDuplicate Records:{duplicate_count}")

print(demand_forecast.shape)

geonode_id    0
sku_id        0
date          0
forecast      0
dtype: int64

Duplicate Records:0
(56364, 4)


In [3]:
# Check for Missing values
org_mapping = pd.read_excel('NIU Data Visualization Competition.xlsx',sheet_name = 'supplier org mapping')
print(org_mapping.isna().sum())

# Check for Duplicate records
duplicate_count = org_mapping.duplicated(keep='first').sum()
print(f"\nDuplicate Records:{duplicate_count}")

geonode_id    0
org_id        0
dtype: int64

Duplicate Records:0


In [5]:
#Merge Capacity and demand forecast dataframes considering all forecast demands(Right Join)
merged_data = pd.merge(capacity, demand_forecast, left_on=['geonode_id','sku_id','start_date'], right_on=['geonode_id','sku_id','date'],how='right')

# Supplier Capacity having no forcast demand
capacity_only = capacity.merge(demand_forecast, left_on=['geonode_id','sku_id','start_date'], right_on=['geonode_id','sku_id','date'],how = 'left',indicator = True)
#print(capacity_only.head(2))
capacity_not_used = capacity_only.loc[capacity_only['_merge']=='left_only']

#Adding "capacity_not_used" rows to "merged_data"
merged_data = pd.concat([merged_data,capacity_not_used])

demand_fulfilled = []
avail=[]

for idx, row in merged_data.iterrows():
    if row["forecast"] <= row["standard_capacity"]:
        availability = row["standard_capacity"] - row["forecast"]
    else:
        availability = row["max_capacity"] - row["forecast"]
    
    if availability < 0:
        demand_fulfilled.append("No")    
    elif ((availability >= 0) & (row["forecast"] != 0)):
        demand_fulfilled.append("Yes")
    elif((availability > 0) & (row["forecast"] == 0))|(row["_merge"] == "left_only"):
        demand_fulfilled.append("Unused capacity")
    elif((availability == 0) & (row["forecast"] == 0)):
        demand_fulfilled.append("ignore")    #both capacity and demand values are 0
    elif row["forecast"] != 0:
        demand_fulfilled.append("Not Available") # Capacity is not available for forecasted demand.
    else:
        demand_fulfilled.append("Ignore")   #Capacity is not available but forecasted demand is 0.
    
    # Append the availability value to the avail list
    avail.append(availability)

# Add the "avail" and "Demand Fullfilled" columns to the merged_data DataFrame
merged_data["avail"] = avail
merged_data["demand_fulfilled"] = demand_fulfilled
merged_data['month_year'] = merged_data['date'].dt.strftime('%b %y')

demand_data = merged_data["demand_fulfilled"].value_counts()
print(demand_data)

demand_fulfilled
Ignore             32463
Yes                14501
Unused capacity     6005
Not Available       4728
No                  1412
ignore                78
Name: count, dtype: int64


In [4]:
from geopy.geocoders import Nominatim

# Function to find location for a geonode_id
def find_location(geonode_id):
    geolocator = Nominatim(user_agent="geo_locator",timeout=100)
    location = geolocator.geocode(geonode_id)
    return location

# Apply find_location function to each geonode_id and create a new column
org_mapping['location'] = org_mapping['geonode_id'].apply(lambda x: find_location(x))


In [13]:
merged_data = pd.merge(merged_data,org_mapping,on="geonode_id",how="left")
merged_data.to_excel("merged_data.xlsx", index=False)